In [11]:
from random import randint as r, choice as ch
import sys
import quizDatabaseHelper 

In [12]:

def printmenu():
    '''print a list of menu items'''
    title ="Welcome to Max's quiz"
    deco = "-"
    menu = '''
1. Multiplication
2. Addition
3. Subtraction
4. Advanced
5. Display All Results
6. Search result by person
7. Quit
    '''
    print(f'{title}\n{deco*len(title)}\n{menu}')


In [13]:
def validator(m):
    while True:
        try:
            if m == "menu":
                choice = int(input("Enter a menu choice (1-7): "))
                if choice in range(1, 8):
                    return choice
                else:
                    print("Your choice is out of range")
            elif m == "ans":
                value = int(input("Enter your answer: "))
                return value
        except ValueError:
            print("Sorry you must enter an integer")

In [14]:
def quiz(s, x):
    """Handles all the quiz operations"""
    score = 0
    divices = []

    if s == "ad":
        divices = [r(1, 3) for _ in range(5)]

    for i in range(5):
        if s == "a" or (s == "ad" and divices[i] == 1):
            num1 = r(1, x)
            num2 = r(1, x)
            correct = num1 + num2
            print(f"{num1} + {num2}")
        elif s == "s" or (s == "ad" and divices[i] == 2):
            num1 = r(1, x)
            num2 = r(1, x)
            correct = num1 - num2
            print(f"{num1} - {num2}")
        elif s == "m" or (s == "ad" and divices[i] == 3):
            num1 = r(1, x)
            num2 = r(1, x)
            correct = num1 * num2
            print(f"{num1} x {num2}")

        answer = validator("ans")
        if answer == correct:
            print("Correct\n")
            score += 1
        else:
            print("Incorrect\n")

    return score


"""def displayAll():
    #prints all scores
    print('All results printed')
    try:
        with open("scores.txt", mode= "r", encoding="utf-8") as scores:
            for line in scores:
                print(line)
    except FileNotFoundError:
        print("No scores avalable yet.")"""

'def displayAll():\n    #prints all scores\n    print(\'All results printed\')\n    try:\n        with open("scores.txt", mode= "r", encoding="utf-8") as scores:\n            for line in scores:\n                print(line)\n    except FileNotFoundError:\n        print("No scores avalable yet.")'

In [15]:
def displayAll():
    readDbdata()


"""def searchByPerson():
    #searches the score by person name
    print('Search initiated')
    search = input("Enter the name: ")
    found= False
    try:
        with open("scores.txt", mode= "r", encoding="utf-8") as scores:
            for line in scores:
                if line.startswith(search+":"):
                    print(line.strip())
                    found=True
        if not found:
            print(f"No results found for {search}.")
    except FileNotFoundError:
        print("No scores avalible yet.")"""


def searchByPerson():
    """Searches for a person's score by name"""
    Nm = input("Who do you want to check: ")
    rt = searchPerson(Nm)
    if rt:
        print(rt)
    else:
        print(f"No results found for {Nm}.")


"""def recordscores(name,score):
    with open("scores.txt", mode= "a", encoding="utf-8") as scores:
        scores.write(f"{name}: {score}\n")"""


def recordscores(name, score):
    """Saves score and name to the database"""
    updateOrAdd(name, score)


def main():
    """Main application function"""
    name = input("Hello, what is your name: ").strip()

    while True:
        printmenu()
        score = -1
        choice = validator("menu")

        if choice == 7:  # Exit the program
            print("Have a nice rest of your day!")
            break
        elif choice == 1:
            print("Multiplication Started!\n\n\n\n")
            score = quiz("m", 10)
            print(f"Correct answers = {score}\n")
        elif choice == 2:
            print("Addition Started!\n\n\n\n")
            score = quiz("a", 10)
            print(f"Correct answers = {score}\n")
        elif choice == 3:
            print("Subtraction Started!\n\n\n\n")
            score = quiz("s", 10)
            print(f"Correct answers = {score}\n")
        elif choice == 4:
            print("Advanced Quiz Started!\n\n\n\n")
            score = quiz("ad", 100)
            print(f"Correct answers = {score}\n")
        elif choice == 5:
            displayAll()
        elif choice == 6:
            searchByPerson()

        if score > -1:
            recordscores(name, score)

In [18]:
from sqlite3 import connect


def dBconnection():
    """Connects and returns a cursor and connection object"""
    conn = connect("quizdb.db")
    cur = conn.cursor()
    return conn, cur


def readDbdata():
    """connect and read records"""
    conn, cur = dBconnection()
    cur.execute("SELECT * FROM tblstudents")
    records = cur.fetchall()
    print(records)


def searchPerson(Nm):
    """connect and read records"""
    conn, cur = dBconnection()
    query = f"""SELECT * FROM tblstudents WHERE name ="{Nm}" """
    cur.execute(
        query,
    )
    records = cur.fetchall()
    return records


def personChecker(Nm):
    conn, cur = dBconnection()
    query = f"""SELECT * FROM tblstudents WHERE name ="{Nm}" """
    cur.execute(
        query,
    )
    results = cur.fetchall()
    return results


def getScores(Nm):
    conn, cur = dBconnection()
    query = f"""SELECT score2, score3 FROM tblstudents WHERE name ="{Nm}" """
    cur.execute(
        query,
    )
    records = cur.fetchall()
    return records


def updateRecord(Nm, S1, S2, S3):
    """Updates a record"""
    conn, cur = dBconnection()
    query = f'''UPDATE tblstudents SET score1={S1}, score2={S2}, score3={S3} WHERE name="{Nm}"'''
    cur.execute(query)
    conn.commit()
    conn.close()


def nextId():
    conn, cur = dBconnection()
    cur.execute("SELECT ID FROM tblstudents")
    records = cur.fetchall()
    NewId = records[-1][0] + 1
    return NewId


def addrecord(Nm, S3):
    """adds new record"""
    conn, cur = dBconnection()
    query = "INSERT INTO tblstudents VALUES(?,?,0, 0, ?)"
    cur.execute(query, (nextId(), Nm, S3))
    conn.commit()
    conn.close()


def updateOrAdd(Nm, S3):
    """Either update an existing record or add a new one"""
    results = personChecker(Nm)
    if results:
        scores = getScores(Nm)
        S2 = scores[0][1]
        S1 = scores[0][0]
        updateRecord(Nm, S1, S2, S3)
    else:
        addrecord(Nm, S3)


scores = [0, 1, 2]
newScore = 7

In [24]:
dBconnection()
readDbdata()
searchPerson("Max")
personChecker("Max")
print(getScores("Max"))
updateRecord("Max", 2, 3,8)
print(getScores("max"))

[(1, 'Max', 1.0, 1, 3), (2, 'Alex', 0.0, 0, 4), (3, 'Charles', 0.0, 0, 7), (4, 'Alice', 0.0, 0, 3), (5, 'luke', 0.0, 0, 2), (6, 'max', 0.0, 0, 5), (7, 'alfie', 0.0, 0, 0)]
[(1, 3)]
[(0, 5)]


In [ ]:
main()